In [1]:
# Variables to be altered
file_name = '21_08_12_CMP.xlsx'
option_file_name  = '21_08_12_Strike.xlsx'
rate = 0.2


In [2]:
import datetime
import pandas as pd
import math

In [3]:
def final_stop(file):
    a = file.split('_')
    final = "20"+a[0]+"-"+a[1]+"-"+a[2]+" 15:25:00"
    return final

def CMPFromExcel(i,df):

    return float(df.iloc[i+1,10])

def TimeFromExcel(i,df):

    return df.iloc[i+1,0]

def CallPremFromExcel(strikePrice,time,df):


    for j in range(df.iloc[-1,14]):
        if(df.iloc[j,6]==str(strikePrice) and df.iloc[j,0]==time):
            return float(df.iloc[j,10])

def getFirstCellC(strikePrice,time,df):

    
    for j in range(df.iloc[-1,14]):
        if(df.iloc[j,6]==str(strikePrice) and df.iloc[j,0]==time):
            return j
    
def CallPremFromExcel2(i,df):

    return float(df.iloc[i,10])

def CallPremFromExcel2High(i,df):
    return float(df.iloc[i,8])
            
def PutPremFromExcel(strikePrice,time,df):

    for j in range(df.iloc[-1,14]):
        if(df.iloc[j,6]==str(strikePrice) and  df.iloc[j,0]==time):
            return float(df.iloc[j,10])

def getFirstCellP(strikePrice,time,df):

    for j in range(df.iloc[-1,14]):
        if(df.iloc[j,6]==str(strikePrice) and df.iloc[j,0]==time):
            return j
def PutPremFromExcel2High(i,df):

    return float(df.iloc[i,8])
            
def PutPremFromExcel2(i,df):

    return float(df.iloc[i,10])

def findJ(startTime,df):
    final_stopTime = final_stop(file_name)
    if(str(startTime).split()==str(final_stopTime).split()):
        exit()
    for j in range(df.iloc[-1,13]):
        if(df.iloc[j,0]==startTime  ):
            return j-1

def today(date):
    date =str(date).split('-')
    month = int(date[1])
    day = int(date[2])
    year = int(date[0])
    a = datetime.date(year,month,day).weekday()
    if a == 0:
        x  = "Monday"

    if a == 1:
        x  ="Tuesday" 

    if a==2:
        x  ="Wednesday" 

    if a==3:
       x ="Thursday" 

    if a==4:
       x ="Friday"

    if a==5:
       x = "Saturday" 

    if a==6: 
        x  = "Sunday"
    return x
    


In [4]:
def start(j,prev_table,total,count,dfc,dfp,dfs):
    count+=1

    initial = j

    cmp = CMPFromExcel(j,dfs)
     
    t = TimeFromExcel(j,dfs)
    day = str(t).split()
    myWeekday = today(day[0])

    stopTime = day[0]+" 15:25:00"
    startTime = day[0]+" 09:20:00"

    zintermediate = (math.floor(cmp/100))*100
    x = cmp%100
    y = math.floor(x)

    if (y>=50):
        strikeprice = zintermediate+100
    else:
        strikeprice = zintermediate
    SP = str(strikeprice)

    iP = getFirstCellP(SP,t,dfp)
    iC = getFirstCellC(SP,t,dfc)

    PE = "CNXBAN "+str(strikeprice)+" PE"
    CE = "CNXBAN "+str(strikeprice)+" CE"



    execPremCall = CallPremFromExcel(SP,t,dfc)
    execPremCall = float(execPremCall)
    execPremPut = PutPremFromExcel(SP,t,dfp)
    execPremPut = float(execPremPut)


    stlp_1C = execPremCall*(1+rate)
    stlp_1P = execPremPut*(1+rate)
    
    putStoplossIsHit = False
    CallStoplossIsHit = False
    trialTime = 0                     #Enter in minutes
    exitPremPut = 0
    exitPremCall = 0



    
    
    while(True):

        
        if(PutPremFromExcel2High(iP,dfp)>=stlp_1P ):
            putStoplossIsHit = True
            
            exitTimePut = TimeFromExcel(j,dfs)
            exitPremPut = min(PutPremFromExcel2High(iP,dfp),stlp_1P+5)
            Real_Put_P_or_L = execPremPut - exitPremPut
  
            break

        if(CallPremFromExcel2High(iC,dfc)>=stlp_1C):
       
            exitPremCall = min(CallPremFromExcel2High(iC,dfc),stlp_1C+5)
            exitTimeCall = TimeFromExcel(j,dfs)
            Real_Call_P_or_L = execPremCall - exitPremCall
  

            break
        if(TimeFromExcel(j,dfs) == stopTime):

            putStoplossIsHit = True
            exitPremCall = CallPremFromExcel2(iC,dfc)
            exitPremPut = PutPremFromExcel2(iP,dfp)
            exitTimeCall = TimeFromExcel(j,dfs)
            Real_Call_P_or_L = execPremCall - exitPremCall
            Real_Put_P_or_L = execPremPut - exitPremPut

            print("FINAL TABLE".center(200,'*'))
            print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")

            print("|    Contract    |         Time         |    Day   |  Quantity  |      Entry Price      |      Current Price      |       Stop Loss        |     Exit Price     |          P&L        |")


            print(prev_table)
            print("|"+PE.center(16)+"|"+TimeFromExcel(j,dfs).center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremPut).center(23)+"|"+"--".center(25)+"|"+str(stlp_1P).center(24)+"|"+str(PutPremFromExcel2(iP,dfp)).center(20)+"|"+str(Real_Put_P_or_L*qty).center(21)+"|")
            print("|"+CE.center(16)+"|"+TimeFromExcel(j,dfs).center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremCall).center(23)+"|"+"--".center(25)+"|"+str(stlp_1C).center(24)+"|"+str(exitPremCall).center(20)+"|"+str(Real_Call_P_or_L*qty).center(21)+"|")
            print(bufferZone)
            print("|"+ "Total ".center(16)+"|"+"".center(22)+"|"+"".center(10)+"|"+"".center(12)+"|"+"".center(23)+"|"+"".center(25)+"|"+"".center(24)+"|"+"".center(20)+"|"+str(((Real_Call_P_or_L+Real_Put_P_or_L)*qty+total)).center(21)+"|")
        

            print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
            print("Day Finished")

            if(count<5 and j+37<dfs.iloc[-1,13] ):
                if(stopTime == final_stopTime):
                    print("ALL DAYS FINISHED")
                    exit()
                t = TimeFromExcel(j+37,dfs)
                day = str(t).split()

                startTime = day[0]+" 09:20:00"

                j = findJ(startTime,dfs)
                start(j,"",0,count,dfc,dfp,dfs)

            else:
                print("ALL DAYS FINISHED")
                exit()


  


        Real_Call_P_or_L = execPremCall - CallPremFromExcel2(iC,dfc)
        Real_Put_P_or_L = execPremPut - PutPremFromExcel2(iP,dfp)

  

        iP =iP+n
        iC = iC+n
        j=j+n     


    if(putStoplossIsHit):





        while(True):
            

            
            if(CMPFromExcel(j,dfs)>=cmp):    # or prem1*(1.1)>=prem
                cmp = CMPFromExcel(j,dfs)
            
                Real_Call_P_or_L = execPremCall - CallPremFromExcel2(iC,dfc)

              
                
                if(CallPremFromExcel2High(iC,dfc)>=stlp_1C):

                    exitPremCall = min(CallPremFromExcel2High(iC,dfc),stlp_1C+5)
                    Real_Call_P_or_L = execPremCall - exitPremCall
                    exitTimeCall = TimeFromExcel(j,dfs)
                    
                 
                    break
                if(TimeFromExcel(j,dfs) == stopTime):

                        exitPremCall = CallPremFromExcel2(iC,dfc)
                        exitTimeCall = TimeFromExcel(j,dfs)

                        Real_Call_P_or_L = execPremCall - exitPremCall
                        Real_Put_P_or_L = execPremPut - exitPremPut
                        print("FINAL TABLE".center(200,'*'))
                        print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")

                        print("|    Contract    |         Time         |    Day   |  Quantity  |      Entry Price      |      Current Price      |       Stop Loss        |     Exit Price     |          P&L        |")


                        print(prev_table)
                        print("|"+PE.center(16)+"|"+exitTimePut.center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremPut).center(23)+"|"+"--".center(25)+"|"+str(stlp_1P).center(24)+"|"+str(exitPremPut).center(20)+"|"+str(Real_Put_P_or_L*qty).center(21)+"|")
                        print("|"+CE.center(16)+"|"+exitTimeCall.center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremCall).center(23)+"|"+"--".center(25)+"|"+str(stlp_1C).center(24)+"|"+str(exitPremCall).center(20)+"|"+str(Real_Call_P_or_L*qty).center(21)+"|")
                        print(bufferZone)
                        print("|"+ "Total ".center(16)+"|"+"".center(22)+"|"+"".center(10)+"|"+"".center(12)+"|"+"".center(23)+"|"+"".center(25)+"|"+"".center(24)+"|"+"".center(20)+"|"+str(((Real_Call_P_or_L+Real_Put_P_or_L)*qty+total)).center(21)+"|")

                        print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
                        print("Day Finished")
                        
                        if(count<5 and j+37<dfs.iloc[-1,13]):
                            if(stopTime == final_stopTime):
                                print("ALL DAYS FINISHED")
                                exit()
                            t = TimeFromExcel(j+37,dfs)
                            day = str(t).split()

                            startTime = day[0]+" 09:20:00"

                            j = findJ(startTime,dfs)


                            start(j,"",0,count,dfc,dfp,dfs)
                        else:
                            print("ALL DAYS FINISHED")
                            exit()


                    
            



            else:
                if(stlp_1C>CallPremFromExcel2(iC,dfc)*(rate+1)):
                    stlp_1C=CallPremFromExcel2(iC,dfc)*(rate+1)
                if(TimeFromExcel(j,dfs) == stopTime):

                    exitPremCall = CallPremFromExcel2(iC,dfc)
                    exitTimeCall = TimeFromExcel(j,dfs)

                    Real_Call_P_or_L = execPremCall - exitPremCall
                    Real_Put_P_or_L = execPremPut - exitPremPut
                    print("FINAL TABLE".center(200,'*'))
                    print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")

                    print("|    Contract    |         Time         |    Day   |  Quantity  |      Entry Price      |      Current Price      |       Stop Loss        |     Exit Price     |          P&L        |")


                    print(prev_table)
                    print("|"+PE.center(16)+"|"++exitTimePut.center(22)+"|"+myWeekday.center(10)++"|"+str(qty).center(12)+"|"+str(execPremPut).center(23)+"|"+"--".center(25)+"|"+str(stlp_1P).center(24)+"|"+str(exitPremPut).center(20)+"|"+str(Real_Put_P_or_L*qty).center(21)+"|")
                    print("|"+CE.center(16)+"|"+exitTimeCall.center(22)+"|"+myWeekday.center(10)++"|"+str(qty).center(12)+"|"+str(execPremCall).center(23)+"|"+"--".center(25)+"|"+str(stlp_1C).center(24)+"|"+str(exitPremCall).center(20)+"|"+str(Real_Call_P_or_L*qty).center(21)+"|")
                    print(bufferZone)
                    print("|"+ "Total ".center(16)+"|"+"".center(22)+"|"+"".center(10)+"|"+"".center(12)+"|"+"".center(23)+"|"+"".center(25)+"|"+"".center(24)+"|"+"".center(20)+"|"+str(((Real_Call_P_or_L+Real_Put_P_or_L)*qty+total)).center(21)+"|")

                    print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
                    print("Day Finished")
                        
                    if(count<5 and j+37<dfs.iloc[-1,13]):
                        if(stopTime == final_stopTime):
                            print("ALL DAYS FINISHED")
                            exit()
                        t = TimeFromExcel(j+37,dfs)
                        day = str(t).split()

                        stopTime = day[0]+" 15:25:00"
                        startTime = day[0]+" 09:20:00"

                        j = findJ(startTime,dfs)

                        
                        start(j,"",0,count,dfc,dfp,dfs)
                    else:
                        print("ALL DAYS FINISHED")
                        exit()

                cmp = CMPFromExcel(j,dfs)
                Real_Call_P_or_L = execPremCall - CallPremFromExcel2(iC,dfc)
             

            j=j+n
            iP =iP+n
            iC = iC+n
            
    else:
   
        while(True):
            


            
            if(CMPFromExcel(j,dfs)<=cmp ): # or prem1*(1.1)>=prem
                cmp = CMPFromExcel(j,dfs)



                Real_Put_P_or_L = execPremPut - PutPremFromExcel2(iP,dfp)

             


                if(PutPremFromExcel2High(iP,dfp)>=stlp_1P):
                    putStoplossIsHit = True
                    exitPremPut =min(PutPremFromExcel2High(iP,dfp),stlp_1P+5)
                    exitTimePut = TimeFromExcel(j,dfs)
                    Real_Put_P_or_L = execPremPut - exitPremPut
                    break

                if(TimeFromExcel(j,dfs) == stopTime):
                        exitPremPut = PutPremFromExcel2(iP,dfp)
                        exitTimePut = TimeFromExcel(j,dfs)
                        Real_Call_P_or_L = execPremCall - exitPremCall
                        Real_Put_P_or_L = execPremPut - exitPremPut
                        print("FINAL TABLE".center(200,'*'))
                        print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")

                        print("|    Contract    |         Time         |    Day   |  Quantity  |      Entry Price      |      Current Price      |       Stop Loss        |     Exit Price     |          P&L        |")


                        print(prev_table)
                        print("|"+PE.center(16)+"|"+exitTimePut.center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremPut).center(23)+"|"+"--".center(25)+"|"+str(stlp_1P).center(24)+"|"+str(exitPremPut).center(20)+"|"+str(Real_Put_P_or_L*qty).center(21)+"|")
                        print("|"+CE.center(16)+"|"+exitTimeCall.center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremCall).center(23)+"|"+"--".center(25)+"|"+"--".center(24)+"|"+str(exitPremCall).center(20)+"|"+str(Real_Call_P_or_L*qty).center(21)+"|")
                        print(bufferZone)
                        print("|"+ "Total ".center(16)+"|"+"".center(22)+"|"+"".center(10)+"|"+"".center(12)+"|"+"".center(23)+"|"+"".center(25)+"|"+"".center(24)+"|"+"".center(20)+"|"+str(((Real_Call_P_or_L+Real_Put_P_or_L)*qty+total)).center(21)+"|")                        
                        print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
                
                        print("Day Finished")
                        
                        if(count<5 and j+37<dfs.iloc[-1,13]):
                            if(stopTime == final_stopTime):
                                print("ALL DAYS FINISHED")
                                exit()
                            t = TimeFromExcel(j+37,dfs)
                            day = str(t).split()

                            stopTime = day[0]+" 15:25:00"
                            startTime = day[0]+" 09:20:00"

                            j = findJ(startTime,dfs)
                            
                            start(j,"",0,count,dfc,dfp,dfs)
                        else:
                            print("ALL DAYS FINISHED")
                            exit()
                            

                    
                    
            else:
                cmp = CMPFromExcel(j,dfs)
                prem = PutPremFromExcel2(iP,dfp)
                if(PutPremFromExcel2(iP,dfp)*(rate+1)<stlp_1P):
                    stlp_1P=PutPremFromExcel2(iP,dfp)*(rate+1)
                if(TimeFromExcel(j,dfs) == stopTime):

                    exitPremCall = CallPremFromExcel2(iC,dfc)
                    exitTimeCall = TimeFromExcel(j,dfs)

                    Real_Call_P_or_L = execPremCall - exitPremCall
                    Real_Put_P_or_L = execPremPut - exitPremPut
                    print("FINAL TABLE".center(200,'*'))
                    print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")

                    print("|    Contract    |         Time         |    Day   |  Quantity  |      Entry Price      |      Current Price      |       Stop Loss        |     Exit Price     |          P&L        |")


                    print(prev_table)
                    print("|"+PE.center(16)+"|"+TimeFromExcel(j,dfs).center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremPut).center(23)+"|"+"--".center(25)+"|"+str(stlp_1P).center(24)+"|"+str(exitPremPut).center(20)+"|"+str(Real_Put_P_or_L*qty).center(21)+"|")
                    print("|"+CE.center(16)+"|"+exitTimeCall.center(22)+"|"+myWeekday.center(10)++"|"+str(qty).center(12)+"|"+str(execPremCall).center(23)+"|"+"--".center(25)+"|"+"--".center(24)+"|"+str(exitPremCall).center(20)+"|"+str(Real_Call_P_or_L*qty).center(21)+"|")
                    print(bufferZone)
                    print("|"+ "Total ".center(16)+"|"+"".center(22)+"|"+"".center(10)+"|"+"".center(12)+"|"+"".center(23)+"|"+"".center(25)+"|"+"".center(24)+"|"+"".center(20)+"|"+str(((Real_Call_P_or_L+Real_Put_P_or_L)*qty+total)).center(21)+"|")

                    print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
                    print("Day Finished")
                        
                    if(count<5 and j+37<dfs.iloc[-1,13]):
                        if(stopTime == final_stopTime):
                            print("ALL DAYS FINISHED")
                            exit()
                        t = TimeFromExcel(j+37,dfs)
                        day = str(t).split()

                        stopTime = day[0]+" 15:25:00"
                        startTime = day[0]+" 09:20:00"

                        j = findJ(startTime,dfs)
                        

                        start(j,"",0,count,dfc,dfp,dfs)                    
                    else:
                        print("ALL DAYS FINISHED")
                        exit()

                
    
                Real_Put_P_or_L = execPremPut - PutPremFromExcel2(iP,dfp)

 
            j=j+n
            iP =iP+n
            iC = iC+n

    session_total = (Real_Call_P_or_L +Real_Put_P_or_L)*qty

    x = "Total = "+str(total)


    

    prev_table = prev_table +"\n"+str(bufferZone)+"|"+PE.center(16)+"|"+str(exitTimePut).center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremPut).center(23)+"|"+"--".center(25)+"|"+str(stlp_1P).center(24)+"|"+str(exitPremPut).center(20)+"|"+str(Real_Put_P_or_L*qty).center(21)+"|\n"+"|"+CE.center(16)+"|"+str(exitTimeCall).center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremCall).center(23)+"|"+"--".center(25)+"|"+str(stlp_1C).center(24)+"|"+str(exitPremCall).center(20)+"|"+str(Real_Call_P_or_L*qty).center(21)+"|\n"
    total  = total+session_total

    if(TimeFromExcel(j,dfs) != str(stopTime)):
        count-=1
        
        if(j==initial):
            start(j+1,prev_table,total,count,dfc,dfp,dfs)
        else:
            start(j+1,prev_table,total,count,dfc,dfp,dfs)


    if(TimeFromExcel(j,dfs) == stopTime):
        
        print("FINAL TABLE".center(200,'*'))
        print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")

        print("|    Contract    |         Time         |    Day   |  Quantity  |      Entry Price      |      Current Price      |       Stop Loss        |     Exit Price     |          P&L        |")


        print(prev_table)
        print("|"+PE.center(16)+"|"++exitTimePut.center(22)+"|"+myWeekday.center(10)++"|"+str(qty).center(12)+"|"+str(execPremPut).center(23)+"|"+"--".center(25)+"|"+str(stlp_1P).center(24)+"|"+str(exitPremPut).center(20)+"|"+str(Real_Put_P_or_L*qty).center(21)+"|")
        print("|"+CE.center(16)+"|"+exitTimeCall.center(22)+"|"+myWeekday.center(10)++"|"+str(qty).center(12)+"|"+str(execPremCall).center(23)+"|"+"--".center(25)+"|"+"--".center(24)+"|"+str(exitPremCall).center(20)+"|"+str(Real_Call_P_or_L*qty).center(21)+"|")
        print(bufferZone)
        print("|"+ "Total ".center(16)+"|"+"".center(22)+"|"+"".center(10)+"|"+"".center(12)+"|"+"".center(23)+"|"+"".center(25)+"|"+"".center(24)+"|"+"".center(20)+"|"+str(((Real_Call_P_or_L+Real_Put_P_or_L)*qty+total)).center(21)+"|")
        print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
                
        print("Day Finished")

        if(count<5 and j+37<dfs.iloc[-1,13] ):
            if(stopTime == final_stopTime):
                print("ALL DAYS FINISHED")
                exit()
        
            t = TimeFromExcel(j+37,dfs)
            day = str(t).split()

            stopTime = day[0]+" 15:25:00"
            startTime = day[0]+" 09:20:00"

            j = findJ(startTime,dfs)


            
            start(j,"",0,count,dfc,dfp,dfs)
 
        else:
            print("ALL DAYS FINISHED")
            exit()



In [5]:


final_stopTime = final_stop(file_name)

dfs = pd.read_excel(file_name,engine='openpyxl',dtype=object,header=None)

lotsize = 25
no_of_lot = 1
qty = no_of_lot*lotsize


n=1
j = 100
t = TimeFromExcel(j,dfs)
day = str(t).split()

stopTime = day[0]+" 15:25:00"
startTime = day[0]+" 09:20:00"

dfs = pd.read_excel(file_name,engine='openpyxl',dtype=object,header=None)
print("dataframes ready")

j = findJ(startTime,dfs)

cmp = CMPFromExcel(j,dfs)

t = TimeFromExcel(j,dfs)
print(t)
bufferZone = "|"+"".center(16)+"|"+"".center(22)+"|"+"".center(10)+"|"+"".center(12)+"|"+"".center(23)+"|"+"".center(25)+"|"+"".center(24)+"|"+"".center(20)+"|"+"".center(21)+"|\n"
dfs = pd.read_excel(file_name,engine='openpyxl',dtype=object,header=None)
dfc = pd.read_excel(option_file_name,sheet_name='Call',engine='openpyxl',dtype=object,header=None)
dfp = pd.read_excel(option_file_name,sheet_name='Put',engine='openpyxl',dtype=object,header=None)


dataframes ready
2021-08-06 09:20:00


In [6]:
start(j,"",0,0,dfc,dfp,dfs)


**********************************************************************************************FINAL TABLE***********************************************************************************************
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|    Contract    |         Time         |    Day   |  Quantity  |      Entry Price      |      Current Price      |       Stop Loss        |     Exit Price     |          P&L        |

|CNXBAN 35900 PE | 2021-08-06 15:25:00  |  Friday  |     25     |         366.1         |            --           |         439.32         |       303.4        |  1567.5000000000011 |
|CNXBAN 35900 CE | 2021-08-06 15:25:00  |  Friday  |     25     |         421.65        |            --           |   505.97999999999996   |       304.65       |        2925.0       |
|                |                      |          |        

**********************************************************************************************FINAL TABLE***********************************************************************************************
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|    Contract    |         Time         |    Day   |  Quantity  |      Entry Price      |      Current Price      |       Stop Loss        |     Exit Price     |          P&L        |

|                |                      |          |            |                       |                         |                        |                    |                     |
|CNXBAN 35900 PE | 2021-08-12 09:21:00  | Thursday |     25     |         105.25        |            --           |         126.3          |       131.3        |  -651.2500000000002 |
|CNXBAN 35900 CE | 2021-08-12 09:28:00  | Thursday |     25 

TypeError: float() argument must be a string or a number, not 'NoneType'